# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [111]:
#Load the csv exported in Part I to a DataFrame
city_data_df = pd.read_csv("Resources/city_data.csv")
#Had to drop the column as the index was not turned off when writing to the CSV file
city_data_df = city_data_df.drop("Unnamed: 0",axis=1) 
city_data_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mehamn,71.0357,27.8492,30.11,86,40,6.91,NO,1634668289
1,nikki,9.9401,3.2108,75.25,86,63,4.70,BJ,1634668364
2,rikitea,-23.1203,-134.9692,74.86,71,5,21.25,PF,1634668135
3,saint george,37.1041,-113.5841,63.54,39,1,3.00,US,1634668135
4,asfi,32.2994,-9.2372,70.84,85,27,6.80,MA,1634668136


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [131]:
#Extract the lat and long from the dataframe
locations = city_data_df[["Lat", "Lng"]]
#Extract humidity for weight on the heatmap
humidity = city_data_df["Humidity"]
fig = gmaps.figure()


#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, opacity=1, point_radius=6, max_intensity=city_data_df["Humidity"].max())

#Add layer
fig.add_layer(heat_layer)
#display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [133]:
#Narrow down the cities to fit weather conditions.
hotel_df = city_data_df.loc[(city_data_df["Max temp"] <= 86) & (city_data_df["Max temp"] >= 75) &
                                    (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 10)] 

hotel_df.head()

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,presidencia roque saenz pena,-26.7852,-60.4388,85.62,17,0,6.93,AR,1634668523
105,bela vista,-22.1089,-56.5211,84.22,31,0,9.22,BR,1634668524
216,illapel,-31.6308,-71.1653,77.94,33,0,9.82,CL,1634668548
310,san luis,-33.2950,-66.3356,81.05,15,0,8.05,AR,1634668569
379,sanchor,24.7553,71.7722,79.99,45,0,8.03,IN,1634668583


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [134]:
#Get the coordinates for the preferred cities

#coordinates = [preffered_cities["Lat"], preffered_cities["Lng"]]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    #Hit the Google Places API for each city's coordinates.
    response = requests.get(base_url, params=params).json()
   
    #Store the first Hotel result into the DataFrame.
    hotel_name = response["results"][0]["name"]
    hotel_df.loc[index, "Hotel Name"] = hotel_name

hotel_df

/Users/b77077/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/b77077/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
99,presidencia roque saenz pena,-26.7852,-60.4388,85.62,17,0,6.93,AR,1634668523,Hotel Aconcagua
105,bela vista,-22.1089,-56.5211,84.22,31,0,9.22,BR,1634668524,Bella Vista Palace Hotel
216,illapel,-31.6308,-71.1653,77.94,33,0,9.82,CL,1634668548,Hotel Londres
310,san luis,-33.2950,-66.3356,81.05,15,0,8.05,AR,1634668569,Huarpes Hotel
379,sanchor,24.7553,71.7722,79.99,45,0,8.03,IN,1634668583,Hotel Amar Palace & AC Restaurant
419,hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634668255,Bungalow 63
444,jiwani,25.0500,61.7417,79.86,76,0,6.58,PK,1634668595,Al Mumtaz Transport Jiwani
478,gwadar,25.1216,62.3254,79.65,73,0,5.99,PK,1634668600,Sadaf Resort
510,doha,25.2867,51.5333,85.98,52,0,9.22,QA,1634668118,Movenpick Hotel Doha


In [135]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [136]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))